In [5]:
import pandas as pd
from tqdm import tqdm
import pandas as pd
import psutil
import multiprocessing
import time
import shutil
import h3
from collections import defaultdict
import pickle
import os

In [3]:
memory_info = psutil.virtual_memory()

print(f"Total Memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available Memory: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Used Memory: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Percentage Used: {memory_info.percent}%")

Total Memory: 125.90 GB
Available Memory: 121.16 GB
Used Memory: 3.74 GB
Percentage Used: 3.8%


In [4]:
file_path = '/disk_5/grid_v2_vm_data/disk_4/pre_processed_data/state_grp_ASSAM/split_train_test_valid_state_ASSAM/781003_full_train.csv'
df = pd.read_csv(file_path)
df.head()

addr_hash addr1                        addr2  \
0  7957c8f2230374e5aa71a7b96c55823a     0                    Chandmari   
1  7957c8f2230374e5aa71a7b96c55823a     0                    Chandmari   
2  421959c90e711ed82f28b202bc042cd0     0  Chandmari Milanpur Guwahati   
3  421959c90e711ed82f28b202bc042cd0     0  Chandmari Milanpur Guwahati   
4  421959c90e711ed82f28b202bc042cd0     0  Chandmari Milanpur Guwahati   

       city  state  pincode country        lat        lng  \
0  Guwahati  Assam   781003   India  26.183486  91.775550   
1  Guwahati  Assam   781003   India  26.183784  91.775953   
2  Guwahati  Assam   781003   India  26.190091  91.781124   
3  Guwahati  Assam   781003   India  26.189986  91.783756   
4  Guwahati  Assam   781003   India  26.190347  91.781784   

                       time  accuracy_level                 original_address  \
0  2017-08-14 12:06:17+0530          10.000                    0 , Chandmari   
1  2022-01-24 14:14:54+0530           6.489                    0 , Chandmari   
2  2017-09-26 14:08:01+0530          32.000  0 , Chandmari Milanpur Guwahati   
3  2017-10-01 15:39:01+0530           5.000  0 , Chandmari Milanpur Guwahati   
4  2017-10-25 15:26:04+0530           5.000  0 , Chandmari Milanpur Guwahati   

                  cleaned_address  lookup_lat  lookup_lng  confidence_score  \
0                     0 chandmari   26.183486   91.775550                 1   
1                     0 chandmari   26.183486   91.775550                 1   
2   0 chandmari milanpur guwahati   26.190347   91.781784                 0   
3   0 chandmari milanpur guwahati   26.190347   91.781784                 0   
4   0 chandmari milanpur guwahati   26.190347   91.781784                 0   

   no_of_delivereies  
0                  2  
1                  2  
2                  3  
3                  3  
4                  3

In [6]:
input_directory_path = '/disk_5/grid_v2_vm_data/disk_4/pre_processed_data/state_grp_ASSAM/split_train_test_valid_state_ASSAM'
output_combined_csv_path = '/disk_4/uddeshya/Reference-uddeshya/Triplet_dataset/State_full_train_CSV/ASSAM_full.csv'
columns_to_keep = ['city', 'pincode', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'no_of_delivereies']

def load_and_process_file(filename):
    file_path = os.path.join(input_directory_path, filename)
    df = pd.read_csv(file_path, usecols=columns_to_keep)
    #df['cleaned_address'] = df['cleaned_address'] + " " + df['city']
    df_filtered = df[df['no_of_delivereies'] >= 1]
    df_filtered = df_filtered.drop_duplicates(subset=['cleaned_address'])
    df_filtered = df_filtered.dropna()
    return df_filtered

def combine_csv_files():
    csv_files = [f for f in os.listdir(input_directory_path) if f.endswith('_full_train.csv')]
    
    combined_df = pd.DataFrame(columns=columns_to_keep)
    for filename in tqdm(csv_files, desc="Loading and processing CSV files"):
        df_filtered = load_and_process_file(filename)
        combined_df = pd.concat([combined_df, df_filtered])
    

    combined_df = combined_df.dropna()
    combined_df.to_csv(output_combined_csv_path, index=False)
    print(f"Combined CSV file saved to {output_combined_csv_path}")

if __name__ == "__main__":
    combine_csv_files()

Loading and processing CSV files: 100%|██████████| 496/496 [08:37<00:00,  1.04s/it]


Combined CSV file saved to /disk_4/uddeshya/Resources/Triplet_dataset/State_full_train_CSV/ASSAM_full.csv
